# Kartenvisualisierung
In diesem Notebook wird demonstriert, wie in Python eine Kartenvisualisierung möglich ist.

Wir stützen uns dabei auf die Tabelle, die im xml_to_table-Skript erstellt wurde.

## Koordinaten hinzufügen
Da unsere Tabelle zwar die Keys der Orte enthält, aber nicht die IDs, müssen wir nochmal die API bemühen, um die Koordinaten zu erhalten.

In [72]:
import pandas as pd

# Einlesen der Tabelle
df = pd.read_csv(input("Geben Sie den Pfad zur CSV-Datei ein:"))

print(df)

        place_text      place_normalized  latitude  longitude
0         Augspurg              Augsburg  48.37154   10.89851
1          Eÿstett             Eichstätt  48.88854   11.19675
2         Augspurg              Augsburg  48.37154   10.89851
3          Eÿstett             Eichstätt  48.88854   11.19675
4          München               München  48.13743   11.57549
5         Augspurg              Augsburg  48.37154   10.89851
6         Augspurg              Augsburg  48.37154   10.89851
7           Betmeß                   NaN       NaN        NaN
8          Newburg  Neuburg an der Donau  48.73218   11.18709
9          Neuburg  Neuburg an der Donau  48.73218   11.18709
10         Schantz                   NaN       NaN        NaN
11         Eÿstett             Eichstätt  48.88854   11.19675
12         trauben                   NaN       NaN        NaN
13      stainbruch                   NaN       NaN        NaN
14      stainbruch                   NaN       NaN        NaN
15      

In [73]:
# wir wollen nicht Orte mehrmals abfragen, auch um den Editions-Server zu schonen
# also machen wir erstmal eine Liste aller einzigartigen Orts-Keys
unique_place_keys = df['place_normalized'].unique().tolist()

print(unique_place_keys)

['Augsburg', 'Eichstätt', 'München', nan, 'Neuburg an der Donau', 'Altmühl', 'Bayern', 'Ingolstadt', 'Pfaffenhofen', 'Deutschland']


## Interaktive Karten mit Python
Im Folgenden verwenden wir das externe Package [*Folium*](https://python-visualization.github.io/folium/latest/index.html), um unsere Orte in einer Karte darzustellen.

In [74]:
%pip install -U folium

Note: you may need to restart the kernel to use updated packages.


In [75]:
import folium

# Eine Tabelle nur mit den relevanten Informationen für die Visualisierung
places_df = df[['place_text', 'latitude', 'longitude']].drop_duplicates()

# die Koordinaten sollten als Zahlenwerte vorliegen, wir stellen das hier nochmal sicher
places_df['latitude'] = places_df['latitude'].astype(float)
places_df['longitude'] = places_df['longitude'].astype(float)
        

# Initialisieren der Karte mit Einstellungen, wo anfangs der Ausschnitt liegen soll
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Unsere Orte auf der Karte anzeigen
for _, row in places_df.iterrows():
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        continue
    folium.Marker(
    location=[row['latitude'], row['longitude']],
    popup=row['place_text'],
    tooltip=row['place_text'],
    icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# Display map
m

### Häufigkeit repräsentieren

In [76]:
# wir zählen wie oft jeder Ort vorkommt
place_counts = df['place_text'].value_counts().rename_axis('place_text').reset_index(name='count')

# Merge zwischen dem reduzierten df und unserer Zählung
places_df = places_df.merge(place_counts, on='place_text', how='left')

print(places_df)

        place_text  latitude  longitude  count
0         Augspurg  48.37154   10.89851      4
1          Eÿstett  48.88854   11.19675      3
2          München  48.13743   11.57549      2
3           Betmeß       NaN        NaN      1
4          Newburg  48.73218   11.18709      1
5          Neuburg  48.73218   11.18709      1
6          Schantz       NaN        NaN      1
7          trauben       NaN        NaN      1
8       stainbruch       NaN        NaN      2
9          Altmühl  48.90000   11.90000      1
10          Baÿern  49.00000   11.50000      1
11           Statt  48.88854   11.19675      1
12         Eÿstatt  48.88854   11.19675      1
13     Eystetensis  48.88854   11.19675      1
14      Jngolstatt  48.76508   11.42372      1
15    Pfaffenhofen  48.53053   11.50500      1
16  guldenen Creuz       NaN        NaN      1
17       Germaniae  51.50000   10.50000      1


In [77]:
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

for _, row in places_df.iterrows():
    # Wir vergrössern die Marker, falls der Ort häufiger vorkommt, aber setzen eine Maximalgrösse von 30
    marker_size = min(10 + row["count"] * 2, 30)
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=marker_size,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=f"{row['place_text']}<br>Vorkommen: {row['count']}",
        tooltip=row['place_text']
    ).add_to(m)

# Anzeigen der Karte
m

ValueError: Location values cannot contain NaNs.

### Visualisierung über Zeit
Beachte hier, dass Folium bzw. TimestampedGeoJson leider keine Datumsangaben vor 1970 anzeigen kann, deshalb behelfen wir uns, indem wir alle Datumsangaben 100 Jahre nach hinten rücken.

In [ ]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# Sicherstellen, dass Datumsangaben und Koordinaten im richtigen Format sind
df['date'] = pd.to_datetime(df['date'])
df['date_fitted'] = df['date'].apply(lambda x: x.replace(year=x.year + 100))

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Alle Einträge ohne Datumsangabe entfernen
df = df.dropna(subset=['date'])

# Zeilen nach Datum sortieren
df = df.sort_values(by='date')

# Unsere Daten in GeoJSON-Format schreiben
geojson_features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]]
        },
        "properties": {
            "times": [row["date_fitted"].isoformat()],  # Convert to timestamp format
            "popup": f"{row['place_name']}<br>Datierung: {row['date']}",
            "tooltip": row["place_name"]
        }
    }
    geojson_features.append(feature)

m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Die GeoJSON-Infos der Karte hinzufügen
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": geojson_features
    },
    period="P1M",  # Jeder Timestep = 1 Monat
    duration="P6M",  # Punkte bleiben für 6 Monate danach noch sichtbar
    auto_play=False  # Auto-Play abschalten
).add_to(m)

# Karte anzeigen
m

# Die Karte als Datei speichern, die wir im Browser öffnen können.
m.save("time_map.html")